In [2]:
!pip install xgboost

   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/124.9 MB 281.8 kB/s eta 0:07:24
   ---------------------------------------- 0.1/124.9 MB 722.1 kB/s eta 0:02:53
   ---------------------------------------- 0.3/124.9 MB 1.5 MB/s eta 0:01:26
   ---------------------------------------- 0.4/124.9 MB 1.6 MB/s eta 0:01:18
   ---------------------------------------- 0.6/124.9 MB 1.9 MB/s eta 0:01:07
   ---------------------------------------- 0.7/124.9 MB 2.1 MB/s eta 0:01:01
   ---------------------------------------- 0.8/124.9 MB 2.0 MB/s eta 0:01:02
   ---------------------------------------- 1.1/124.9 MB 2.4 MB/s eta 0:00:51
   ---------------------------------------- 1.3/124.9 MB 2.6 MB/s eta 0:00:47
   -------

In [56]:
# import the dependemcies
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [57]:
import warnings
warnings.filterwarnings('ignore')

In [58]:
# Create dataframe
df = pd.read_csv("customer.csv", sep = ";")


In [59]:
df.head()

,account length,location code,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,"churn,,,,"
0,128,415,3824657,no,yes,25,265,45,17,110,197,87,"244,7",91,"11,01",10,3,"2,7",1,"0,"
1,107,415,3717191,no,yes,26,162,27,17,123,196,103,"254,4",103,"11,45","13,7",3,"3,7",1,0
2,137,415,3581921,no,no,0,243,41,10,114,121,110,"162,6",104,"7,32","12,2",5,"3,29",0,0
3,84,408,3759999,yes,no,0,299,51,5,71,62,88,"196,9",89,"8,86","6,6",7,"1,78",2,0
4,75,415,3306626,yes,no,0,167,28,13,113,148,122,"186,9",121,"8,41","10,1",3,"2,73",3,0


In [60]:
df.shape

(3333, 20)

In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3333 entries, 0 to 3332
Data columns (total 20 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   account length                       3333 non-null   int64 
 1   location code                        3333 non-null   int64 
 2   user id                              3333 non-null   int64 
 3   credit card info save                3333 non-null   object
 4   push status                          3333 non-null   object
 5   add to wishlist                      3333 non-null   int64 
 6   desktop sessions                     3333 non-null   int64 
 7   app sessions                         3333 non-null   int64 
 8   desktop transactions                 3333 non-null   int64 
 9   total product detail views           3333 non-null   int64 
 10  session duration                     3333 non-null   int64 
 11  promotion clicks                     3333 n

In [62]:
df["location code"].unique()

array([415, 408, 510], dtype=int64)

In [63]:
# Changing the data type of the location code column
df["location code"] = df["location code"].astype(str)

In [64]:
df["credit card info save"].unique()

array(['no', 'yes'], dtype=object)

In [65]:
df["credit card info save"].unique()

array(['no', 'yes'], dtype=object)

In [66]:
# Changing the values from yes and no to 1 and 0
df["credit card info save"] = df["credit card info save"].replace({"yes": 1, "no": 0})
df["push status"] = df["push status"].replace({"yes": 1, "no": 0})

In [67]:
df["avg order value"] = df["avg order value"].astype(str).str.replace(',', '.').astype(float)
df["discount rate per visited products"] = df["discount rate per visited products"].astype(str).str.replace(',', '.').astype(float)
df["product detail view per app session"] = df["product detail view per app session"].astype(str).str.replace(',', '.').astype(float)
print(df.dtypes)
df["avg order value"] = df["avg order value"].replace(',', '.', regex=True).astype(float)
df["avg order value"] = df["avg order value"].astype(str).fillna("").str.replace(',', '.').astype(float)


account length                           int64
location code                           object
user id                                  int64
credit card info save                    int64
push status                              int64
add to wishlist                          int64
desktop sessions                         int64
app sessions                             int64
desktop transactions                     int64
total product detail views               int64
session duration                         int64
promotion clicks                         int64
avg order value                        float64
sale product views                       int64
discount rate per visited products     float64
product detail view per app session    float64
app transactions                         int64
add to cart per session                 object
customer service calls                   int64
churn,,,,                               object
dtype: object


In [68]:
# Handling the categorical values using get_dummies
df = pd.get_dummies(df, columns = ["location code"]) 

In [69]:
# Dropping the column user id
df.columns

Index(['account length', 'user id', 'credit card info save', 'push status',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls', 'churn,,,,',
       'location code_408', 'location code_415', 'location code_510'],
      dtype='object')

In [70]:
df.head()

,account length,user id,credit card info save,push status,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,...,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls,"churn,,,,",location code_408,location code_415,location code_510
0,128,3824657,0,1,25,265,45,17,110,197,...,91,11.01,10.0,3,"2,7",1,"0,",False,True,False
1,107,3717191,0,1,26,162,27,17,123,196,...,103,11.45,13.7,3,"3,7",1,0,False,True,False
2,137,3581921,0,0,0,243,41,10,114,121,...,104,7.32,12.2,5,"3,29",0,0,False,True,False
3,84,3759999,1,0,0,299,51,5,71,62,...,89,8.86,6.6,7,"1,78",2,0,True,False,False
4,75,3306626,1,0,0,167,28,13,113,148,...,121,8.41,10.1,3,"2,73",3,0,False,True,False


In [72]:
# Normalizing some column values
# Replace commas with periods and convert to float
df[cols_to_scale] = df[cols_to_scale].replace(',', '.', regex=True).astype(float)

cols_to_scale = ['account length',
       'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls']
scaler = Normalizer()
scaled_data =scaler.fit_transform(df[cols_to_scale])
scaled_df = pd.DataFrame(scaled_data, index = df.index, columns = cols_to_scale)

In [73]:
scaled_df.head()

,account length,add to wishlist,desktop sessions,app sessions,desktop transactions,total product detail views,session duration,promotion clicks,avg order value,sale product views,discount rate per visited products,product detail view per app session,app transactions,add to cart per session,customer service calls
0,0.275142,0.053739,0.569631,0.096730,0.036542,0.236451,0.423461,0.187011,0.525995,0.195609,0.023667,0.021496,0.006449,0.005804,0.002150
1,0.252755,0.061417,0.382676,0.063779,0.040157,0.290551,0.462991,0.243307,0.600944,0.243307,0.027047,0.032362,0.007087,0.008740,0.002362
2,0.345945,0.000000,0.613611,0.103531,0.025251,0.287867,0.305543,0.277766,0.410589,0.262615,0.018484,0.030807,0.012626,0.008308,0.000000
3,0.208327,0.000000,0.741543,0.126484,0.012400,0.176086,0.153765,0.218247,0.488327,0.220727,0.021973,0.016369,0.017361,0.004415,0.004960
4,0.205041,0.000000,0.456559,0.076549,0.035541,0.308929,0.404615,0.333534,0.510963,0.330800,0.022992,0.027612,0.008202,0.007464,0.008202


In [74]:
# Merge the normalized dataframe with the original dataframe
df = df.drop(cols_to_scale, axis = 1)
df = pd.merge(df, scaled_df, left_index=True, right_index=True)
df.columns

Index(['user id', 'credit card info save', 'push status', 'churn,,,,',
       'location code_408', 'location code_415', 'location code_510',
       'account length', 'add to wishlist', 'desktop sessions', 'app sessions',
       'desktop transactions', 'total product detail views',
       'session duration', 'promotion clicks', 'avg order value',
       'sale product views', 'discount rate per visited products',
       'product detail view per app session', 'app transactions',
       'add to cart per session', 'customer service calls'],
      dtype='object')

In [81]:
# Train-test splitting
X = df.drop("churn,,,,", axis=1)
y = df["churn,,,,"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
 # Check what values exist
y_train = y_train.astype(str).str.replace(r'\D+', '', regex=True).astype(int)
y_test = y_test.astype(str).str.replace(r'\D+', '', regex=True).astype(int)
print(y_train.unique()) 

[0 1]


In [77]:
# Printing the shapes of X_train, X_test, y_train and y_test
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(2233, 21) (1100, 21) (2233,) (1100,)


In [82]:
# Build the model
xgb_cl = xgb.XGBClassifier()
xgb_cl.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [83]:
# Predicting on the test data (X_test)
preds = xgb_cl.predict(X_test)

In [84]:
# Print the model accuracy
acc = accuracy_score(y_test, preds)
print("Model Accuracy for Test Dataset:", acc)

Model Accuracy for Test Dataset: 0.9227272727272727


In [85]:
# hyperparameter tuning
param_grid = {
    "max_depth": [5],
    "learning_rate": [0, 0.01, 0.05, 0.1],
    "gamma": [1, 5, 10],
    "scale_pos_weight": [2, 5, 10, 20],
    "subsample":[1],
    "colsample_bytree": [1]
}
xgb_cl2 = xgb.XGBClassifier(objective="binary:logistic")
grid_cv = GridSearchCV(xgb_cl2, param_grid, n_jobs=-1, cv=3, scoring="roc_auc")
_ = grid_cv.fit(X_train, y_train)
print("The Best Score:", grid_cv.best_score_)
print("The Best Params:", grid_cv.best_params_)

The Best Score: 0.8704339782457916
The Best Params: {'colsample_bytree': 1, 'gamma': 1, 'learning_rate': 0.1, 'max_depth': 5, 'scale_pos_weight': 2, 'subsample': 1}


In [86]:
final_cl = xgb.XGBClassifier(
    **grid_cv.best_params_, objective="binary:logistic"
)
grid_final = final_cl.fit(X_train, y_train)
preds = grid_final.predict(X_test)
acc = accuracy_score(y_test, preds)
print("Accuracy of the Final Model:", acc)

Accuracy of the Final Model: 0.9245454545454546
